Конвертация тегов pymorphy. В рамках представленных мною текстов, кажется, такая конвертация ничего не портит, но мб она не всегда будет работать не на всех текстах.

In [1]:
def pymorphy_convert(tag):
  to_verb = ['INFN', 'PRTF', 'PRTS', 'GRND']
  other = ['ADJF', 'ADJS', 'ADVB', 'NUMR']
  if tag == "COMP":
    return 'ADJ'
  if tag == 'NPRO':
    return 'PRON'
  if tag == 'PRED':
    return 'ADV'
  if tag == 'PRCL':
    return 'PART'
  if tag == 'PREP':
    return 'ADP'
  if tag in to_verb:
    return 'VERB'
  if tag in other:
    return tag[:(len(tag) - 1)]
  return tag

То же для spacy и natasha (у них кажется просто одинаковый набор - я их упрощаю, чтобы проще было сравнивать с pymorphy)

In [2]:
def spacy_natasha_convert(tag):
  if 'CONJ' in tag:
    return 'CONJ'
  if tag == 'PROPN':
    return 'NOUN'
  if tag == "AUX":
    return "VERB"
  return tag

In [3]:
def tokenize(text):
  text = word_tokenize(text)
  text = [w.lower() for w in text if w.isalpha()]
  return text

Функция на случай если какие-то теги записались не как строки, а как другие структуры, оставшиеся после морф анализаторов

In [4]:
def make_str(tags):
  new_tags = []
  for tag in tags:
    if type(tag) is not str:
      new_tags.append(str(tag))
    else:
      new_tags.append(tag)
  return new_tags

После того как я вручную выявила id токенов где программа не ошиблась, а просто разная разметка - сложно было унифицировать такие случаи (+ их было немного и размечать такое вручную нормально)

In [45]:
def fix_tags(ids, y, df):
  new_y = []
  for id in range(len(y)):
    if id in ids:
      ndf = df['POS']
      pos = list(ndf[df['id'] == id])[0]
      new_y.append(pos)
    else:
      new_y.append(y[id])
  return new_y

In [6]:
text = ''
with open("corpora.txt", 'r', encoding="utf-8") as f:
  for line in f:
    text += line

In [7]:
import csv
import string
from string import punctuation

In [8]:
import pandas as pd

Так выглядит табличка с id токенов, предложений и pos-разметкой токенов. За основу своей разметки я брала разметку udpipe, потому что мы с ней много работали на курсе лингвистических данных, и это было проще всего.

In [9]:
df = pd.read_csv('standard.tsv', delimiter='\t')

In [67]:
df

,id,id_sent,token,POS
0,0,0,небесное,ADJ
1,1,0,светило,NOUN
2,2,0,было,VERB
3,3,0,закрыто,VERB
4,4,0,дождевыми,ADJ
...,...,...,...,...
220,220,27,команда,NOUN
221,221,27,победила,VERB
222,222,27,благодаря,ADP
223,223,27,помощи,NOUN


Делаю массив из столбика POS для дальнейшего подсчета accuracy

In [10]:
y_true = list(df['POS'])

Вот текст. Сюда включены всякие примеры грамматической омонимии: светило (гл) - светило (сущ), благодаря (гл) - благодаря (предлог). Наверное, сложный случай - умение отличать сравнительную стрепень прилагательного от аналогичной формы у наречия. В корпус так же вошли случаи, где надо понять, где краткое прил, а где причастие

In [11]:
text

'Небесное светило было закрыто дождевыми тучами, а вчера оно ярко и ласково нам светило. Если бы мыло приходило по утрам ко мне в кровать и само меня бы мыло, хорошо бы это было. Мы ели ершей у ели. Скорей, сестра, на рыб взгляни, попались на крючок они. В ведерко руку окуни, не бойся: это окуни! Я любила печь пироги, пока в доме перекладывали печь. Подплывая к берегу, я берегу свою книгу, чтобы она не намокла от воды. В гостиной присутствующие ограничены условностями этикета. Эти люди консервативны и ограниченны. Средства на ремонт изысканы, теперь у нас есть деньги, чтобы осуществить мечту. Все ее платья весьма изысканны и выполнены из качественных материалов. Марина умна и воспитанна, потому что она воспитана коллективом! Это существительное образовано от глагола. Мои братья образованны и работают в хороших фирмах. Петр ведет себя странно. Глория смотрела тепло и ласково. Мишино поведение странно. Сумка легче чемодана. Сегодня солнце светит ярче. Озеро ближе, чем река. Этот способ д

In [12]:
!pip install  pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [14]:
import nltk
from nltk.tokenize import word_tokenize

In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
tokens = tokenize(text)

In [17]:
y_pymorphy = []

Я беру первый разбор, потому что если рассматривать все, то с высокой долей вероятности pymorphy угадает, потому что он просто перечисляет все возможные разборы для этой формы, что кажется читерством в нашей задаче.

In [18]:
for token in tokens:
  ana = morph.parse(token)
  pos = pymorphy_convert(ana[0].tag.POS)
  y_pymorphy.append(pos)

In [19]:
y_pymorphy = make_str(y_pymorphy)

In [20]:
!pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [22]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [23]:
doc = Doc(text)

In [24]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [25]:
y_natasha = []

In [26]:
for i in doc.tokens:
  pos = spacy_natasha_convert(i.pos)
  if pos != "PUNCT":
    y_natasha.append(pos)

In [27]:
len(y_natasha)

225

In [28]:
y_natasha = make_str(y_natasha)

In [29]:
import spacy

In [30]:
import sys
!{sys.executable} -m spacy download ru_core_news_sm

2022-10-11 19:32:33.470264: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [31]:
y_spacy = []

In [32]:
nlp = spacy.load("ru_core_news_sm")
doc = nlp(text)
for token in doc:
    pos = spacy_natasha_convert(token.pos_)
    if pos != "PUNCT":
      y_spacy.append(pos)

In [33]:
len(y_spacy)

225

In [34]:
y_spacy = make_str(y_spacy)

In [35]:
import sklearn
from sklearn.metrics import accuracy_score

После того, как получили ответы от всех морф анализаторов, посчитаем точность их ответов.

In [36]:
accuracy_score(y_true, y_pymorphy)

0.84

In [37]:
accuracy_score(y_true, y_natasha)

0.8977777777777778

In [38]:
accuracy_score(y_true, y_spacy)

0.9288888888888889

Тут я вручную выяснила, в каких токенах полученный ответ не является неправильным, это просто сложный случай, где возможны разные разборы - их тяжело привести автоматически. Поэтому я просто заменила "неправильные" ответы программ на свои ответы, чтобы не учитывать их в подсчете accuracy.

In [39]:
ids_pymorphy = [69, 83, 101, 120, 124, 155, 214, 219]

In [40]:
ids_natasha = [15, 27, 30, 120]

In [41]:
ids_spacy = [15, 27, 30, 119]

In [46]:
y_pymorphy = fix_tags(ids_pymorphy, y_pymorphy, df)

Accuracy улучшилось везде, но все равно spacy справился лучше всех.

In [47]:
accuracy_score(y_true, y_pymorphy)

0.8755555555555555

In [50]:
y_natasha = fix_tags(ids_natasha, y_natasha, df)

In [51]:
accuracy_score(y_true, y_natasha)

0.9155555555555556

In [52]:
y_spacy = fix_tags(ids_spacy, y_spacy, df)

In [53]:
accuracy_score(y_true, y_spacy)

0.9466666666666667

Продолжение этой домашки смотрите в файле hw1_new.ipynb, потому что я вставила в код первой дз улучшения, о которых спрашивалось в задании